<a href="https://colab.research.google.com/github/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [53]:
# Write your code here
# open and read file
import nltk
nltk.download('punkt')
with open("cleaned_review_content.txt", "r", encoding='utf-8') as file:
  data = file.read()




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [54]:
# (1) Count the frequency of all the N-grams (N=3)
import pandas
from textblob import TextBlob
tri_grams=TextBlob(data).ngrams(3)
#print(tri_grams)
hist=dict()
for gram in tri_grams:
  gram=tuple(gram)
  hist[gram]=hist.get(gram,0)+1

pandas.DataFrame.from_dict(hist, orient = 'index', columns=['tri-grams frequency'])



,tri-grams frequency
"(person, saw, hype)",1
"(saw, hype, claim)",1
"(hype, claim, masterpiec)",1
"(claim, masterpiec, overreact)",1
"(masterpiec, overreact, overblown)",1
...,...
"(feel, like, hey)",1
"(like, hey, least)",1
"(hey, least, cinematographi)",1
"(least, cinematographi, decent)",1


In [55]:
#(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.
import pandas
monty = TextBlob (data)
bi_grams=TextBlob(data).ngrams(2)
#print(bi_grams)
hist={}
for gram in bi_grams:
  gram=tuple(gram)
  hist[gram]=hist.get(gram,0)+1

bigr_prob={}
for key, val in hist.items():
  bigr_prob[key]=val/monty.words.count(key[0])
   
pandas.DataFrame.from_dict(bigr_prob, orient = 'index', columns=['bigrams probabilities'])





,bigrams probabilities
"(person, saw)",0.052632
"(saw, hype)",0.111111
"(hype, claim)",0.090909
"(claim, masterpiec)",0.500000
"(masterpiec, overreact)",0.062500
...,...
"(movi, look)",0.004464
"(like, hey)",0.018519
"(hey, least)",0.500000
"(cinematographi, decent)",0.055556


In [56]:
#(3) Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).
# Extract all the noun phrases 
import pandas
import nltk
nltk.download('brown')
monty=TextBlob(data)
print(monty.noun_phrases)

hist={}
for np in monty.noun_phrases:
  hist[np]=hist.get(np,0)+1

max_refreq = max([val for val in hist.values()])

re_prob={} 
for key, val in hist.items():
  re_prob[key] = val/max_refreq

pandas.DataFrame.from_dict (re_prob, orient = 'index', columns = ['noun-phrases relative probabilities'])





[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
['hype claim masterpiec overreact overblown excit anoth joker base film', 'bit pretenti trailer', 'wrong massiv achiev cinema', 'rare day age cgi nonsens reboot', 'reboot sort standalon origin tale impecc', 'finish echo resembl', 'joker origin', 'comic past joaquin', 'sweat cri everi drop magnific dedic', 'heath ledger', 'proud undoubtedli', 'sinc heath joker', 'brilliant bleak set tone palpabl', 'film place blown', 'everi audienc member awestruck wit film', 'transport charact world', 'believ hype', 'rever transcend masterpiec cinema everi movi', 'impact joaquin', 'scenographi brillianc grotesqu haunt cringi', 'time mesmer wont blink eye', 'serious funni moment emot rollercoast sometim multipl emot poppingup timethi', 'typic actionriddl', 'superhero movi', 'proper psycholog thrillerdrama singl', 'alon isol truli relat', 'sorri charact lot peopl', 'encourag violenc truli movi e

,noun-phrases relative probabilities
hype claim masterpiec overreact overblown excit anoth joker base film,0.2
bit pretenti trailer,0.2
wrong massiv achiev cinema,0.2
rare day age cgi nonsens reboot,0.2
reboot sort standalon origin tale impecc,0.2
...,...
movi lot,0.2
touch subject alway leav audienc determin film,0.2
nothingthi smart movi power commentari todd phillip,0.2
kind movi look,0.2


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [57]:
# (1) To build the documents-terms weights (tf*idf) matrix bold text.
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
voca=[word for word in set(data.split())]
corpus = {v: k for v, k in enumerate(data.splitlines(),1)}
tfidf=TfidfVectorizer(vocabulary=voca, max_features=50, lowercase=True, analyzer='word', stop_words='english', ngram_range=(1,3))
tfs=tfidf.fit_transform(corpus.values())
feature_names= tfidf.get_feature_names()
corpus_index=[n for n in corpus]
df=pandas.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
df.head(10)



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96
sold,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.165743,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
say,0.060514,0.0,0.0,0.0,0.101351,0.0,0.072784,0.0,0.051213,0.0,0.099387,0.0,0.0,0.0,0.0,0.051656,0.085332,0.0,0.082818,0.0,0.0,0.0,0.0,0.089027,0.0,0.0,0.083021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062687,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07985,0.0,0.071839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.043292,0.062999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057845,0.027823,0.0,0.0,0.0,0.0,0.0,0.060622,0.0,0.203797,0.0
understood,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
enviro,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.258049,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
appreci,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.102491,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
told,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.094094,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
spend,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062524,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
exponenti,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.157406,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
well,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00

In [58]:
#(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using cosine similarity.
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
from textblob import Word
query="I'm not a big fan of action movie, but this movie is REALLY a masterpiece!!! It is a perfect combination of acting, storyline, soundtrack and everything. It deserves an Oscar of this year."
# preprocess query
query=query.lower()
query=re.sub('[^\w\s]', '', query)
stop=stopwords.words('english')
query= " ".join(x for x in query.split() if x not in stop)
st=PorterStemmer()
query=" ".join([st.stem(word) for word in query.split()])
query= " ".join([Word(word).lemmatize() for word in query.split()])
print(query)

# add query to review corpus
with open("cleaned_review_content.txt", "a", encoding='utf-8') as file:
  file.write('\n')
  file.write(query)
with open("cleaned_review_content.txt", "r", encoding='utf-8') as file:
  data_query = file.read()






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
im big fan action movi movi realli masterpiec perfect combin act storylin soundtrack everyth deserv oscar year


In [59]:
# extract features representing all reviews and the query
voca=[word for word in set(data_query.split())]
corpus = {v: k for v, k in enumerate(data_query.splitlines(),1)}
tfidf=TfidfVectorizer(vocabulary=voca, max_features=50, lowercase=True, analyzer='word', stop_words='english', ngram_range=(1,3))
tfs=tfidf.fit_transform(corpus.values())
feature_names= tfidf.get_feature_names()
corpus_index=[n for n in corpus]
df=pandas.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
df.head(10)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98
sold,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.165841,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
say,0.060725,0.0,0.0,0.0,0.101657,0.0,0.073049,0.0,0.051369,0.0,0.099719,0.0,0.0,0.0,0.0,0.051843,0.085665,0.0,0.083213,0.0,0.0,0.0,0.0,0.089259,0.0,0.0,0.083266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062885,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.080093,0.0,0.072035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.043409,0.063212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058003,0.027900,0.0,0.0,0.0,0.0,0.0,0.060837,0.0,0.204381,0.0,0.0,0.0
understood,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
enviro,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.257682,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
appreci,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.102378,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
told,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.094025,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
spend,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062507,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
exponenti,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.157173,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
well,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000

In [60]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfs[97:98], tfs)

array([[0.0397678 , 0.02123359, 0.03498415, 0.10321069, 0.0522275 ,
        0.        , 0.07113124, 0.06589575, 0.06158508, 0.15004643,
        0.05834909, 0.04128308, 0.13580138, 0.02796549, 0.12705704,
        0.10994023, 0.08908244, 0.04393836, 0.20035077, 0.02563219,
        0.06953574, 0.08771371, 0.07229987, 0.02023869, 0.06987863,
        0.        , 0.10022974, 0.02312905, 0.024335  , 0.        ,
        0.01327614, 0.        , 0.11168094, 0.05344723, 0.03064317,
        0.        , 0.04814448, 0.11688727, 0.10079334, 0.05908771,
        0.08390759, 0.11202301, 0.09971239, 0.06507228, 0.06272688,
        0.07353799, 0.06134536, 0.        , 0.05811851, 0.09098585,
        0.02476683, 0.05895476, 0.04446746, 0.03947487, 0.06190766,
        0.0349183 , 0.04455616, 0.05600782, 0.0451392 , 0.01867348,
        0.04342927, 0.        , 0.05848356, 0.03906139, 0.12328115,
        0.08069159, 0.06906844, 0.03816821, 0.07966127, 0.10262982,
        0.09430183, 0.02843214, 0.06137505, 0.05

# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: 
https://github.com/HuyenNguyenHelen/Huyen_INFO5731_Spring2020/blob/master/Huyen_labeled_review_sentiments.csv 
